#2021 기계학습 기말 프로젝트

##학번 / 이름:   2017320114 최재원

# A. 실험 내용에 대한 전체 요약

#### 사용한 데이터: Adult


#### 1) 개요
선정한 데이터에 대한 설명
- 이 Dataset은 UCI에서 제공한 Dataset으로 1994년에 미국에서 성인을 대상으로 한 인구조사이다. 이 데이터로 여러 feature들을 통해 연소득이 $50,000이 넘는지 넘지 않는지 인구별로 결과를 제시해 준다.

데이터 URL
- url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

Feature 설명
- Age: 나이
- workclass: 고용 형태
- fnlwgt: 개인의 가중치
- education: 학력
- education-num: 개인 교육 수준
- marital-status: 결혼 상태
- occupation: 직업 종
- relationship: 가족 관계
- race: 인종
- sex: 성별
- capital-gain: 이익
- capital-loss: 손실
- hours-per-week: 주간 노동 시간
- native-country: 국적
- income: 소득(label)
---


#### 2) 목표
전처리를 진행한 후 완료된 adult dataset을 가지고 인구의 feature에 따라서 소득(income)이 연 $50,000이 넘는지 안 넘는지 예측(predict)할 것이다.


---


#### 3)모델
1. Decision Tree
2. Random Forest(Ensemble Method)
3. Multilayer Perceptron(MLP)


---


#### 4) 실험 목차
1. 데이터 전처리 진행(Data Preprocessing)
  - MLP의 경우 MLP를 위해서 따로 데이터 전처리 진행(one-hot encoding 추가)

2. 0-R을 통한 Baseline 성능 분석

3. Split Data
  - model을 훈련시킬 train set, model 훈련 과정에서 성능을 평가할 validation set, 그리고 각각의 모델의 최종 성능을 평가할 test set으로 나눈다.

4. *Decision Tree, Random Forest, MLP*에 대해서 Build and Train Model
  - 3개의 모델에 대해 parameter를 조정해서 train data로 훈련

5. 결과 분석 및 모델 선정
  - 각각의 모델들의 parameter pruning model을 validation data set을 이용해서 성능 분석


---

#### 5) 최종 모델 선정
1. 각각의 최적의 Parameter가 정해진 모델들에 대해서 test set으로 최종 성능(accuracy) 비교

2. 최종적으로 Random Forest 모델 선정

In [4]:
# 1. import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(suppress = True)
import warnings

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

#B. 데이터 전처리(Data Preprocessing)

In [56]:
#columns 이름 지정
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
data = pd.read_csv(url, header = None, names = columns, index_col = False)

#missing value drop
data = data.dropna()

In [57]:
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [58]:
data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

In [59]:
#feature의 obect형 전부 정수형으로 인코딩
le = LabelEncoder()
data.workclass = le.fit_transform(data.workclass)
data.education = le.fit_transform(data.education)
data.marital_status = le.fit_transform(data.marital_status)
data.occupation = le.fit_transform(data.occupation)
data.relationship = le.fit_transform(data.relationship)
data.race = le.fit_transform(data.race)
data.sex = le.fit_transform(data.sex)
data.native_country = le.fit_transform(data.native_country)
data.income = le.fit_transform(data.income)

In [60]:
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


##C. 0-R rule 분석 
- 본격적으로 시작하기 이전에, 다른 모든 feautre들을 무시하고 오직 label로만 분류를 해서 앞으로 모델의 baseline이 될 성능을 분석한다.

In [81]:
data['income'].value_counts(sort=True)

0    24720
1     7841
Name: income, dtype: int64

In [43]:
print('income이 $50,000보다 낮은 것"으로 예측하는 ZeroR 모델의 정확도: ', 24720 / (24720 + 7841))
print('income이 $50,000보다 높은 것"으로 예측하는 ZeroR 모델의 정확도: ', 7841 / (24720 + 7841))

income이 $50,000보다 낮은 것"으로 예측하는 ZeroR 모델의 정확도:  0.7591904425539756
income이 $50,000보다 높은 것"으로 예측하는 ZeroR 모델의 정확도:  0.2408095574460244


# D. 실험 방법 & Split Data

#### 1. 실험 방법
- Data를 각각 Train set 과 Test set으로 split한다.

- Split 후 Train data를 train data와 validation data로 split을 해줘서 모델을 훈련 시켜준다.

- 모델을 생성한다.
  1. Decision Tree
  2. Random Forest(Ensemble Method)
  3. Multilayer Perceptron(MLP)

- Parameter에 변경을 주면서 최적의 parameter를 찾을 수 있게 훈련시킨다.

- 최적의 Parameter를 가진 각각의 모델을 선정한다.
  - 이때 validation data를 사용해서 성능을 보여주게 한다. 각각의 모델별로 최적의 parameter를 가진 모델을 고른다.

- Test set에 대하여 각각의 최종 모델의 성능을 평가한다. 

In [71]:
# split data
y = data[['income']]
x = data.drop(columns = ['income'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = .4)


# E. 모델 생성 & Parameter 조정 과정

## 1. Decision Tree
- 기본 default 값으로 진행해 보았다.


In [87]:
# DecisionTreeClassifier: Default
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
print(classifier.score(x_val, y_val))

0.8065153010858835


- parameter pruning을 각각의 parameter별로 진행해 본다.

In [88]:
#max_leaf_nodes 조정
classifier_max_leaf_nodes = DecisionTreeClassifier(max_leaf_nodes=25) 
classifier_max_leaf_nodes.fit(x_train, y_train)

#max_depth 조정
classifier_max_depth = DecisionTreeClassifier(max_depth=6)
classifier_max_depth.fit(x_train, y_train)

#min_samples_split 조정
classifier_min_samples_split = DecisionTreeClassifier(min_samples_split=60) #min_samples_split
classifier_min_samples_split.fit(x_train, y_train)

#min_samples_leaf 조정
classifier_min_samples_leaf = DecisionTreeClassifier(min_samples_leaf = 30) #min_samples_leaf
classifier_min_samples_leaf.fit(x_train, y_train)

#모든 parameter 조정 값 합산
classifier_mix = DecisionTreeClassifier(max_leaf_nodes = 25, max_depth = 6, min_samples_leaf = 30, min_samples_split = 60)
classifier_mix.fit(x_train, y_train)




DecisionTreeClassifier(max_depth=6, max_leaf_nodes=25, min_samples_leaf=30,
                       min_samples_split=60)

- 각각의 pruning 모델들을 validation data를 통해서 성능(accuracy)를 출력해본다.

In [89]:
print("Accuracy non-pruning:", classifier.score(x_val, y_val))
print("Accuracy max_leaf_node:", classifier_max_leaf_nodes.score(x_val, y_val))
print("Accuracy max_depth:", classifier_max_depth.score(x_val, y_val))
print("Accuracy min_samples_split:", classifier_min_samples_split.score(x_val, y_val))
print("Accuracy min_samples_leaf:", classifier_min_samples_leaf.score(x_val, y_val))
print("Accuracy final:", classifier_mix.score(x_val, y_val))

Accuracy non-pruning: 0.8065153010858835
Accuracy max_leaf_node: 0.8543380497970824
Accuracy max_depth: 0.8479763079960513
Accuracy min_samples_split: 0.8424920478227487
Accuracy min_samples_leaf: 0.8484150488099156
Accuracy final: 0.8474278819787211


1. 총 5개의 Decision Tree를 만들어서 Parameter 값에 변화를 주었다. 
  - overfitting을 막기 위해 max_depth 그리고 max_leaf_nodes 값을 default 값인 무제한에서 제한을 두었다. 그 결과 pruning을 진행하지 않은 Tree 보다 좋은 성능을 보였다.

  - min_samples_leaf 그리고 min_samples_split은 작게 설정할수록 분할하는 노드가 많아져서 overfitting 될 가능성이 높기 때문에 defalut 값인 1과 2에서 각각 증가 시켜주었다. 그 결과 pruning을 하지 않은 모델보다 accuracy에서 좋은 성능을 보였다.

  - 마지막으로 각각의 최적의 Parameter 값들을 한번 조합시켜서 최적의 성능을 보이는지 확인하기 위해서 모델을 만들어 봤다. 그 결과 overfitting이 되어서 근소한 차이로 더 낮은 성능을 보였다.

2. 결론: 결과적으로 pruning을 진행한 모델 트리들이 pruning을 안한 tree보다 좋은 성능을 보였고, 그 중 max_leaf_node를 25로 조정한 모델 트리가 가장 좋은 성능(accuracy)를 보였다.

## 2. Random Forest(Ensemble Method)
- 기본 default 값으로 진행해 보았다.

In [117]:
forest_default = RandomForestClassifier(random_state=42)
forest_default.fit(x_train, y_train)
print(forest_default.score(x_val, y_val))

0.857847976307996


- parameter pruning을 각각의 parameter별로 진행해 본다.

In [118]:
#max_leaf_nodes 조정
forest_max_leaf_nodes = RandomForestClassifier(max_leaf_nodes=25)
forest_max_leaf_nodes.fit(x_train, y_train)

#max_depth 조정
forest_max_depth = RandomForestClassifier(max_depth=6)
forest_max_depth.fit(x_train, y_train)

#min_samples_split 조정
forest_min_samples_split = RandomForestClassifier(min_samples_split=100)
forest_min_samples_split.fit(x_train, y_train)

#min_samples_leaf 조정
forest_min_samples_leaf = RandomForestClassifier(min_samples_leaf = 30)
forest_min_samples_leaf.fit(x_train, y_train)

#모든 parameter 조정 값 합산
forest_mix = RandomForestClassifier(max_leaf_nodes = 25, max_depth = 6, min_samples_leaf = 30, min_samples_split = 100, n_estimators = 200)
forest_mix.fit(x_train, y_train)

#n_estimator 조정
forest_n_estimator = RandomForestClassifier(n_estimators=200)
forest_n_estimator.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200)

- 각각의 pruning 모델들을 validation data를 통해서 성능(accuracy)를 출력해본다.

In [119]:
print("Accuracy non-pruning:", forest_default.score(x_val, y_val))
print("Accuracy max_leaf_node:", forest_max_leaf_nodes.score(x_val, y_val))
print("Accuracy max_depth:", forest_max_depth.score(x_val, y_val))
print("Accuracy min_samples_split:", forest_min_samples_split.score(x_val, y_val))
print("Accuracy min_samples_leaf:", forest_min_samples_leaf.score(x_val, y_val))
print("Accuracy n_estimator:", forest_n_estimator.score(x_val, y_val))
print("Accuracy final:", forest_mix.score(x_val, y_val))


Accuracy non-pruning: 0.857847976307996
Accuracy max_leaf_node: 0.8517056049138971
Accuracy max_depth: 0.8544477350005484
Accuracy min_samples_split: 0.86190632883624
Accuracy min_samples_leaf: 0.8572995502906657
Accuracy n_estimator: 0.8568608094768015
Accuracy final: 0.8506087528792365


1. 총 7개의 Random forest를 만들어서 Parameter 값에 변화를 주었다. 
  - Random Forest의 아아디어는 잘 작동하되 서로 다른 방향으로 과대적합된 트리를 많이 만들면 그 결과를 평균냄으로써 과대적합된 양을 줄이고 결과적으로 트리 모델의 예측 성능을 유지하면서 overfitting을 줄이는 것이다.

  - 따라서 이전 Decision 트리 모델에서 찾은 최적의 parameter 값으로 Random Forest에서 parameter pruning을 진행해 보았다. 그 결과, min_samples_split만이 기본 모델보다 좋은 성능을 보였고, 나머지 pruning parameter 모델들은 근소한 차이로 기본 모델보다 낮은 성능을 보였다. 


2. 결론: 기본적으로 랜덤 포레스트는 성능이 매우 뛰어나고 매개변수 튜닝을 많이 하지 않아도 잘 작동한다. 따라서 일반적으로는 기본값을 쓰는 것이 가장 좋은 방법이다. 하지만 이번 실험에서는 그래도 min_samples_split parameter를 100으로 조정한 모델이 가장 좋은 성능(accuracy)를 보였다.

###Feature Selection(Random Forest)
- 추가로 Random Forest Classifier를 사용해서 feature의 중요도를 계산해 본다.

- SelectFromModel을 사용해서 특정 threshold를 넘는 특성만 선택하도록 한다.

In [72]:
from sklearn.feature_selection import SelectFromModel
# MLPClassifier와 SelectFromModel을 이용하여 중요한 특성을 선택합니다.
select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state = 42), threshold='median')

select.fit(x_train, y_train)
x_train_l1 = select.transform(x_train)
x_val_l1 = select.transform(x_val)
print("x_train.shape:", x_train.shape)
print("x_train_l1.shape:", x_train_l1.shape)
print("x_val.shape:", x_val.shape)
print("x_val_l1.shape:", x_val_l1.shape)

x_train.shape: (13675, 14)
x_train_l1.shape: (13675, 7)
x_val.shape: (9117, 14)
x_val_l1.shape: (9117, 7)


#### Feature Selection을 진행한 Random Forest 모델 결과 출력

In [74]:
feature_selection = RandomForestClassifier(min_samples_split = 100)
feature_selection.fit(x_train_l1, y_train)
print("score: ", feature_selection.score(x_val_l1, y_val))

score:  0.8533508829658879


#### 결론
- 기존에 선정했던 최적의 parameter pruning을 반영한 Random Forest에 feature selection을 진행한 결과 기존의 Random Forest의 최적의 모델의 결과보다 낮게 나왔다. 이는 Random Forest의 feature를 반으로 제거했기 때문에 accuracy가 낮게 나온 것으로 추측이 된다.

## 3. MLP(Multi-Layer-Perceptron)
- 이전에 전처리 했던 데이터와는 다르게 전처리를 진행하기 위해 다시 새로 데이터를 url을 통해서 가져온다.

In [75]:
#columns 이름 지정
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
data_mlp = pd.read_csv(url, header = None, names = columns, index_col = False)

#missing value drop
data_mlp = data_mlp.dropna()

In [76]:
data_mlp.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
income            object
dtype: object

### MLP 데이터 전처리
- feature 중에 object data type들은 one-hot encoding을 해준다

- 'Sex' feature는 정수형으로 인코딩 해준다.

- 'income' feature도 정수형으로 인코딩 해준다.

In [77]:
#'sex', 'income' 정수형으로 인코딩
le = LabelEncoder()
data_mlp.sex = le.fit_transform(data_mlp.sex)
data.income = le.fit_transform(data.income)
#나머지 feature one-hot encoding 진행
object_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'native_country']
def onehot(data, feature):
  '''
  data의 feature column을 one hot으로 변환해줍니다.
  data: pandas DataFrame
  feature: string, 데이터 프레임의 column 이름
  '''
  return pd.concat([data, pd.get_dummies(data[feature], prefix=feature)], axis=1).drop([feature], axis=1)

for col in object_columns:
  data_mlp = onehot(data_mlp, col)

In [78]:
data_mlp.dtypes

age                                int64
fnlwgt                             int64
education_num                      int64
sex                                int64
capital_gain                       int64
                                   ...  
native_country_ Thailand           uint8
native_country_ Trinadad&Tobago    uint8
native_country_ United-States      uint8
native_country_ Vietnam            uint8
native_country_ Yugoslavia         uint8
Length: 108, dtype: object

###MLP data split
- 데이터를 train set, validation set, test set으로 나눠준다.

In [79]:
# split data
y_mlp = data_mlp[['income']]
x_mlp = data_mlp.drop(columns = ['income'])

x_train_mlp, x_test_mlp, y_train_mlp, y_test_mlp = train_test_split(x_mlp, y_mlp, test_size = .3)

x_train_mlp, x_val_mlp, y_train_mlp, y_val_mlp = train_test_split(x_train_mlp, y_train_mlp, test_size = .4)


- 기본 default 값으로 진행해 보았다.

In [49]:
mlp = MLPClassifier()
mlp.fit(x_train_mlp, y_train_mlp)
print(mlp.score(x_val_mlp, y_val_mlp))

0.7770099813535154


###parameter pruning을 각각의 parameter 조합으로 진행해 본다.

-조정할 Parameter는 아래와 같다. 

  - hidden_layer_sizes: The ith element represents the number of neurons in the ith hidden layer
    - tuple, length = n_layers -2, default=(100,)
  
  - activation: Activation function for the hidden layer
    - {'identity','logistic','tanh','relu'}, default = 'relu'

  - solver: The solver for weight optimization
    - {'lbfgs', 'sgd', 'adam'}, default = 'adam'

참고문헌: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [50]:
#hidden layer size: 100, 2 | activation: identity
mlp1 = MLPClassifier(hidden_layer_sizes = (100, 2), max_iter = 1000, activation = 'identity', solver = 'adam')
mlp1.fit(x_train_mlp, y_train_mlp)

#hidden layer size: 200, 2 | activation: tanh
mlp2 = MLPClassifier(hidden_layer_sizes = (200, 2), max_iter = 1000, activation = 'tanh', solver = 'adam')
mlp2.fit(x_train_mlp, y_train_mlp)

#hidden layer size: 200, 2 | activation: logistic
mlp3 = MLPClassifier(hidden_layer_sizes = (200, 2), max_iter = 1000, activation = 'logistic', solver = 'adam')
mlp3.fit(x_train_mlp, y_train_mlp)

#hidden layer size: 200, 4 | activation: logistic | max_iter = 2000
mlp4 = MLPClassifier(hidden_layer_sizes = (200, 4), max_iter = 2000, activation = 'logistic', solver = 'adam')
mlp4.fit(x_train_mlp, y_train_mlp)

#hidden layer size: 200, 4 | activation: identity | max_iter = 2000
mlp5 = MLPClassifier(hidden_layer_sizes = (200, 4), max_iter = 2000, activation = 'identity', solver = 'adam')
mlp5.fit(x_train_mlp, y_train_mlp)
         
#hidden layer size: 200, 4 | activation: tanh
mlp6 = MLPClassifier(hidden_layer_sizes = (200, 4), max_iter = 1000, activation = 'tanh', solver = 'adam')
mlp6.fit(x_train_mlp, y_train_mlp)

MLPClassifier(activation='tanh', hidden_layer_sizes=(200, 4), max_iter=1000)

In [51]:
print('default MLP: ', mlp.score(x_val_mlp, y_val_mlp))
print('hidden layer size: 100, 2 | activation: identity: ', mlp1.score(x_val_mlp, y_val_mlp))
print('hidden layer size: 200, 2 | activation: tanh: ', mlp2.score(x_val_mlp, y_val_mlp))
print('hidden layer size: 200, 2 | activation: logistic', mlp3.score(x_val_mlp, y_val_mlp))
print('hidden layer size: 200, 4 | activation: logistic | max_iter: 2000:  ', mlp4.score(x_val_mlp, y_val_mlp))
print('hidden layer size: 200, 4 | activation: identity | max_iter: 2000:  ', mlp5.score(x_val_mlp, y_val_mlp))
print('hidden layer size: 200, 4 | activation: tanh', mlp6.score(x_val_mlp, y_val_mlp))

default MLP:  0.7770099813535154
hidden layer size: 100, 2 | activation: identity:  0.7303937698804431
hidden layer size: 200, 2 | activation: tanh:  0.763628386530657
hidden layer size: 200, 2 | activation: logistic 0.7617637380717341
hidden layer size: 200, 4 | activation: logistic | max_iter: 2000:   0.7626412196994625
hidden layer size: 200, 4 | activation: identity | max_iter: 2000:   0.7921465394318307
hidden layer size: 200, 4 | activation: tanh 0.7669189426346386


총 7개의 MLP를 만들어서 Parameter 값에 변화를 주었다. 이번에는 다른 모델들과는 다르게 각각의 parameter가 아닌 parameter들의 조합으로 진행을 했다. Optimization 방법을 adam으로 고정해 놓은 채, activation 그리고 hidden layer의 크기를 조정해서 최적의 모델을 찾는 실험을 총 7개의 모델을 기준으로 진행했다.

1. Hidden layer size: hidden layer size는 어떠한 activation parameter에서도 (100,2) 보다는 (200,2)가 더 좋은 성능을 보였다. 따라서 (200,2)를 고르게 되었고, 추후에 (200,4)로 layer size를 높여서 진행해본 결과, (200,4)가 더 좋은 accuracy를 보여서 (200,4)로 parameter를 조정하게 되었다.

2. activation: activation은 여러가지 조합들을 hidden layer 와 맞춰서 진행해 본 결과, 모든 activation이 hidden layer size를 높이면서 조정해 나갈 때 좋은 성능을 보여줬다.

3. 결론: 최종적으로 MLP 모델은 hidden layer parameter (200, 4), activation: identity, 그리고 max_iter: 2000으로 pruning을 해주었을 때 가장 좋은 성능을 보였다.

#F. 최종 모델 선정
- Test set을 이용해서 각각의 모델의 최종 성능(accuracy)를 비교한다.

In [84]:
#Decision Tree
Decision_Tree_Final = DecisionTreeClassifier(max_leaf_nodes=25) 
Decision_Tree_Final.fit(x_train, y_train)

#Random Forest
Random_Forest_Final = RandomForestClassifier(min_samples_split=100)
Random_Forest_Final.fit(x_train, y_train)

#MLP
MLP_Final = MLPClassifier(hidden_layer_sizes = (200, 4), max_iter = 2000, activation = 'identity', solver = 'adam')
MLP_Final.fit(x_train_mlp, y_train_mlp)


MLPClassifier(activation='identity', hidden_layer_sizes=(200, 4), max_iter=2000)

In [85]:
in#max_leaf_node parameter 조정한 DECESION TREE
print("Optimal Decision Tree Accuracy: ", Decision_Tree_Final.score(x_test, y_test))

#min_samples_split 조정한 RANDOM FOREST
print("Optimal Random Forest Accuracy:", Random_Forest_Final.score(x_test, y_test))

#hidden layer size: 200, 4 | activation: identity | max_iter = 2000로 조정한 MLP
#(MLP는 전처리 과정이 다르기 때문에 다른 test data 사용)
print('Optimal MLP Accuracy ', MLP_Final.score(x_test_mlp, y_test_mlp))


Optimal Decision Tree Accuracy:  0.8493192752584706
Optimal Random Forest Accuracy: 0.8545398710205753
Optimal MLP Accuracy  0.7903572525335244


## **"Adult Data에 대해서 연소득이(income)이 $50,000 이상인지 이하인지 예측하는 모델은 Random Forest Model로 선택을 했다. 최종 성능 *0.855*을 보였다."**